# 0. Import Data

In [1]:
%reload_ext tensorboard
import pandas as pd
import datetime
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Churn.csv')

In [3]:
# Elimina Churn y customer id de las variables independientes
X = pd.get_dummies(df.drop(['Churn', 'Customer ID'], axis=1))
# Transformar la variable Churn Yes a 1 y No a 0 
y = df['Churn'].apply(lambda x: 1 if x=='Yes' else 0)

In [4]:
#Separa X e Y en dos muestras una para entrenar el modelo y otro para testear modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [5]:
y_train.head()

946     1
6191    1
1218    0
2826    0
2765    0
Name: Churn, dtype: int64

# 1. Import Dependencies

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

# 2. Build and Compile Model

In [7]:
#Crear modelo secuencial
model = Sequential()
#Agregar capas
model.add(Dense(units=32, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

2022-08-25 19:07:08.765337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

# 3. Fit, Predict and Evaluate

In [ ]:
#Creamos un directorio con archivo log que va guardando el ajuste del modelo despues de cada epoca
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
%tensorboard --logdir log_dir
#Entrenamos el modelo con la muestra de entrenamiento
model.fit(X_train, y_train, epochs=500, batch_size=32,validation_data=(X_test,y_test),callbacks=[tensorboard_callback])


In [10]:
#realizamos predicciones de Y con el modelo entrenado usando los X (variables independientes) de testeo
y_hat = model.predict(X_test,callbacks=[tensorboard_callback])
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

45/45 [==============================] - 1s 5ms/step


In [11]:
#Calculamos la eficacia del modelo comparando los Y reales de la muestra de testeo versus los Y predichos por el modelo
accuracy_score(y_test, y_hat)

0.7778566359119943

Vemos que el 77,8% de las predicciones fueron acertadas

# 4. Saving and Reloading

In [12]:
model.save('tfmodel')

INFO:tensorflow:Assets written to: tfmodel/assets


In [13]:
del model 

In [14]:
model = load_model('tfmodel')